In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import sys
sys.path.append("../")
from CustomFunctions import tfidf_preprocessing, BERT_Embeddings

C:\Users\Prash\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel("../Data/LabeledText.xlsx")
data.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,negative
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.txt,This is my Valentine's from 1 of my nephews. I...,positive
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,neutral
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,positive


In [3]:
encode = LabelEncoder()
data['LABEL'] = encode.fit_transform(data['LABEL'])
data.head()

,File Name,Caption,LABEL
0,1.txt,How I feel today #legday #jelly #aching #gym,0
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,0
2,100.txt,This is my Valentine's from 1 of my nephews. I...,2
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,1
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,2


In [4]:
data['Caption Length'] = data['Caption'].apply(len)
data['Hashtags'] = data['Caption'].str.extractall(r'#(\w+)').groupby(level=0)[0].apply(' '.join)
data['Hashtags'] = data['Hashtags'].fillna('')
data['Total Hashtags'] = data['Hashtags'].str.split(' ').apply(lambda x: len(x) if x[0] != '' else 0)
data['Hashtags'].replace('', np.nan, inplace=True)

data.head()

,File Name,Caption,LABEL,Caption Length,Hashtags,Total Hashtags
0,1.txt,How I feel today #legday #jelly #aching #gym,0,45,legday jelly aching gym,4
1,10.txt,@ArrivaTW absolute disgrace two carriages from...,0,99,disgraced,1
2,100.txt,This is my Valentine's from 1 of my nephews. I...,2,117,NaN,0
3,1000.txt,betterfeelingfilms: RT via Instagram: First da...,1,93,powerless,1
4,1001.txt,Zoe's first love #Rattled @JohnnyHarper15,2,42,Rattled,1


In [ ]:
tweet_tfidf_train, tweet_tfidf_test, y_train, y_test = tfidf_preprocessing(data['Caption'], data['LABEL'], stratify=False, shuffle=False)

In [ ]:
print(y_train[:10])
print(data['LABEL'].iloc[:10])

In [ ]:
from scipy.sparse import vstack

tweet_tfidf = vstack([tweet_tfidf_train, tweet_tfidf_test])
tweet_tfidf.shape, tweet_tfidf_train.shape, tweet_tfidf_test.shape

In [ ]:
y = np.hstack([y_train, y_test])
y.shape

In [ ]:
type(y)

In [ ]:
type(data['Hashtags'].iloc[5])

In [ ]:
from nltk.tokenize import word_tokenize

a = data['Hashtags'].iloc[2]
a
# word_tokenize(a)

In [ ]:
hashtags_tfidf_train, hashtags_tfidf_test, y_train, y_test = tfidf_preprocessing(data['Hashtags'], data['LABEL'], stratify=False, shuffle=False)

In [ ]:
hashtags_tfidf = vstack([hashtags_tfidf_train, hashtags_tfidf_test])
hashtags_tfidf.shape

In [ ]:
data['Total Hashtags'].unique()

In [11]:
data['Hashtags'].fillna("", inplace=True)

In [12]:
embed = BERT_Embeddings(data, 'LABEL', 0.2, 'bert-base-uncased', stratify=False, shuffle=False)

In [13]:
embed.encode_data("Hashtags")
embed.create_hidden_states(128)

Map: 100%|██████████| 974/974 [00:49<00:00, 19.84 examples/s]


In [14]:
embed.create_train_test_set()
a,b,c,d = embed.get_train_test()

In [16]:
e = np.hstack([c,d])

In [17]:
data['LABEL'].iloc[10:20], e[10:20]

(10    0
 11    0
 12    2
 13    0
 14    0
 15    1
 16    1
 17    0
 18    0
 19    0
 Name: LABEL, dtype: int32,
 array([0, 0, 2, 0, 0, 1, 1, 0, 0, 0], dtype=int64))